In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import DataFrame
from datetime import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/train3/procedures.csv
/kaggle/input/train3/payer_transitions.csv
/kaggle/input/train3/patients.csv
/kaggle/input/train4/providers.csv
/kaggle/input/train4/supplies.csv
/kaggle/input/train4/payers.csv
/kaggle/input/train2/encounters.csv
/kaggle/input/train2/medications.csv
/kaggle/input/train2/observations.csv
/kaggle/input/testzip/test/encounters.csv
/kaggle/input/testzip/test/procedures.csv
/kaggle/input/testzip/test/providers.csv
/kaggle/input/testzip/test/payer_transitions.csv
/kaggle/input/testzip/test/supplies.csv
/kaggle/input/testzip/test/medications.csv
/kaggle/input/testzip/test/payers.csv
/kaggle/input/testzip/test/patients.csv
/kaggle/input/testzip/test/conditions.csv
/kaggle/input/testzip/test/devices.csv
/kaggle/input/testzip/test/allergies.csv
/kaggle/input/testzip/test/observations.csv
/kaggle/input/testzip/test/careplans.csv
/kaggle/input/testzip/test/imaging_studies.csv
/kaggle/input/testzip/test/organizations.csv
/kaggle/input/testzip/test/immunizations.

In [ ]:
testpatients = pd.read_csv('/kaggle/input/testzip/test/patients.csv')
tobservation = pd.read_csv('/kaggle/input/testzip/test/observations.csv')
tprocedures = pd.read_csv('/kaggle/input/testzip/test/procedures.csv')
timmunizations = pd.read_csv('/kaggle/input/testzip/test/immunizations.csv')
tconditions = pd.read_csv('/kaggle/input/testzip/test/conditions.csv')
tcareplans = pd.read_csv('/kaggle/input/testzip/test/careplans.csv')

In [46]:
#testobservation = pd.read_csv('/kaggle/input/testzip/test/observations.csv')
careplans = pd.read_csv('/kaggle/input/train1/careplans.csv')
immunizations = pd.read_csv('/kaggle/input/train1/immunizations.csv')
patients = pd.read_csv('/kaggle/input/train3/patients.csv')
observations = pd.read_csv('/kaggle/input/train2/observations.csv')
encounters = pd.read_csv('/kaggle/input/train2/encounters.csv')
conditions = pd.read_csv('/kaggle/input/train1/conditions.csv')
patients = pd.read_csv('/kaggle/input/train3/patients.csv')
procedures = pd.read_csv('/kaggle/input/train3/procedures.csv')
medications = pd.read_csv('/kaggle/input/train2/medications.csv')  

In [ ]:
# try test set hwoever there is no 'Hospital admission for isolation (procedure)' in DESCRIPTION, 
#but Emergency hospital admission for asthma;  Emergency room admission (procedure) might be important!

#encounters = pd.read_csv('/kaggle/input/testzip/test/encounters.csv')
#encounters['START']= encounters['START'].astype('datetime64[ns]')
#encounters['STOP']= encounters['STOP'].astype('datetime64[ns]')
#encounters['NUM_DAYS_HOPS'] = (encounters['STOP']-encounters['START']).astype('timedelta64[D]').astype(float)
#encounters= encounters[encounters.START.dt.year == 2020]
#encounters = encounters[encounters.NUM_DAYS_HOPS != 0.0]
#encounters= encounters[encounters.DESCRIPTION == Hospital admission] 
#encounters= encounters[encounters.ENCOUNTERCLASS == 'inpatient' ]
#encounters.DESCRIPTION.value_counts()  #Emergency hospital admission for asthma;  Emergency room admission (procedure) 


In [47]:
## identify neg/pos covid-19 status --> 'COVID_STAT'
neg_covid_id = observations[(observations.CODE == '94531-1')
                                & (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
pos_covid_id = conditions[(conditions.CODE == 840539006)].PATIENT.unique()
pos_covid_id = DataFrame(pos_covid_id, columns = ['Id'])
neg_covid_id = DataFrame(neg_covid_id, columns = ['Id'])
patients['COVID_STAT'] = 1  # ''
patients.loc[patients.Id.isin(pos_covid_id.Id),'COVID_STAT'] = 0

In [52]:
# find death after 2020 and alive. --> 'DEATH_2020'
patients['DEATHDATE']= patients['DEATHDATE'].astype('datetime64[ns]')
patients['BIRTHDATE']= patients['BIRTHDATE'].astype('datetime64[ns]')
patients['Decease_2020'] = 0
patients.loc[patients.DEATHDATE.dt.year == 2020,'Decease_2020'] = 1
patients.loc[patients.DEATHDATE.dt.year < 2020,'Decease_2020'] = 0.1 

In [133]:
# alive & decease from covid 
patients['Target_AD'] = 1.0
#patients.loc[patients.Id.isin(pos_covid_id.Id),'Target_AD'] = 0.5                                     ?????? ignore positive alive status 
DEATH_patients = DataFrame(patients[~(patients.DEATHDATE.isna())])
pos_covid_id.set_index('Id', inplace=True)
DEATH_patients.set_index('Id', inplace=True)
idx1 = pd.Index(DEATH_patients.index)
idx2 = pd.Index(pos_covid_id.index)
a = idx1.intersection(idx2)
deceased_covid19_ids = DataFrame(a)
patients.loc[patients.Id.isin(deceased_covid19_ids.Id),'Target_AD'] = 0.0
patients = pd.merge(patients,
                    deceased_covid19_ids,
                    how = 'left',
                    on = 'Id')
# age 
patients['DEATHDATE'] = patients['DEATHDATE'].fillna(np.datetime64('NOW'))
patients['AGE'] = (patients['DEATHDATE'] - patients['BIRTHDATE'].astype('M8[D]')).astype('timedelta64[Y]').astype(float)
#patients['AGE'] = (np.datetime64('NOW') - patients['BIRTHDATE'].astype('M8[D]')).astype('timedelta64[Y]').astype(float)

patients.head(3) 

In [134]:
patients.Target_AD.value_counts()

1.0    112391
0.0      5568
Name: Target_AD, dtype: int64

In [58]:
# total hospitalized num of days for every patients 
from datetime import datetime
encounters['START']= encounters['START'].astype('datetime64[ns]')
encounters['STOP']= encounters['STOP'].astype('datetime64[ns]')
total_hospitalized_days = DataFrame(encounters[['PATIENT','START','STOP']])
total_hospitalized_days['NUM_DAYS_HOPS'] = (total_hospitalized_days['STOP']-total_hospitalized_days['START']).astype('timedelta64[D]').astype(float)
total_hospitalized_days = total_hospitalized_days.groupby(['PATIENT']).sum()
patients = pd.merge(patients,
                   total_hospitalized_days,
                   how = 'left',
                   left_on = 'Id',
                   right_on= 'PATIENT')
patients.head(1)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,COVID_STAT,Target_AD,Decease_2020,AGE,NUM_DAYS_HOPS
0,93fa6213-73a1-4cce-b408-9986a4145801,1989-09-05,2020-07-02 23:26:50,999-87-8276,NaN,NaN,NaN,Roland928,Wolff180,NaN,...,NaN,42.398277,-71.289383,6520.55,0.0,1,1.0,0.0,30.0,0.0


In [59]:
hospitalized_ids

,Id
0,26e5d262-6cdc-4274-a5cd-d7fd439e35f9
1,90031c21-e8a4-416e-b60b-23e871ee62dc
2,c58ae02e-235e-4db2-a9c3-6926e9c0c5fa
3,0e07d4b4-8c0a-4cd0-a439-8f3b96ab884d
4,ef272f3c-6cbd-401c-97b5-db5de666b6e5
...,...
20397,184d0311-e033-4c97-8d66-3809ca91be94
20398,acff780f-ccd1-4cef-ba8f-40eb70972098
20399,65d3785a-6150-4b86-8842-c40ca654e3c4
20400,497bf8a5-2c19-4149-9a2e-f256c8165a7a


In [71]:
# identify hospitolized covid-19 status
hospitalized_ids = encounters[(encounters.REASONCODE == 840539006)
                         & (encounters.CODE ==1505002)].PATIENT.unique()
hospitalized_ids = DataFrame(hospitalized_ids,columns=['Id'])  # 20402 rows × 1 columns

covid_hospitalized_days = DataFrame(encounters[encounters.PATIENT.isin(hospitalized_ids.Id)])
## compute days 
covid_hospitalized_days = covid_hospitalized_days[['PATIENT','START','STOP','DESCRIPTION','ENCOUNTERCLASS']]
covid_hospitalized_days['START']= covid_hospitalized_days['START'].astype('datetime64[ns]')
covid_hospitalized_days['STOP']= covid_hospitalized_days['STOP'].astype('datetime64[ns]')
covid_hospitalized_days['Covid_NUM_DAYS_HOPS'] = (covid_hospitalized_days['STOP']-covid_hospitalized_days['START']).astype('timedelta64[D]').astype(float)
## (drop days before 2020); delete days = 0.0; only remain isolation --> 
covid_hospitalized_days = covid_hospitalized_days[covid_hospitalized_days.START.dt.year == 2020]
covid_hospitalized_days = covid_hospitalized_days[covid_hospitalized_days.Covid_NUM_DAYS_HOPS != 0.0]
covid_hospitalized_days = covid_hospitalized_days.groupby(['PATIENT']).sum()
#covid_hospitalized_days = covid_hospitalized_days[covid_hospitalized_days.DESCRIPTION == 'Hospital admission for isolation (procedure)' ]
covid_hospitalized_days # DESCRIPTION,ENCOUNTERCLASS values are unique; all START after 2020 

# !!important finding: encounters['DESCRIPTION'] = Hospital admission for isolation (procedure) is an important feature since num = covid_hospitalized_days rows
# however feature are are not the same, if you want to identify which feature is important you have to check whether it is exist in testsets. 
# solutions : adding more weights on DESCRIPTION,ENCOUNTERCLASS,NUM_DAYS_HOPS
patients = pd.merge(patients,
                   covid_hospitalized_days,
                   how = 'left',
                   left_on = 'Id',
                   right_on= 'PATIENT')
patients['Covid_NUM_DAYS_HOPS'] = patients['Covid_NUM_DAYS_HOPS'].fillna(0.0)
#patients.drop(['PATIENT_x','START_x','STOP_x','PATIENT_y','START_y','STOP_y'], axis=1, inplace=True)
patients.head(3)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,COVID_STAT,Target_AD,Decease_2020,AGE,NUM_DAYS_HOPS,Covid_NUM_DAYS_ICU,Covid_NUM_DAYS_HOPS
0,93fa6213-73a1-4cce-b408-9986a4145801,1989-09-05,2020-07-02 23:26:50,999-87-8276,NaN,NaN,NaN,Roland928,Wolff180,NaN,...,-71.289383,6520.55,0.00,1,1.0,0.0,30.0,0.0,0.0,0.0
1,53bbfa41-5cab-422e-a8a7-8fa3ae55ed71,1997-09-25,2020-07-02 23:24:55,999-96-1808,S99962046,X54333499X,Mr.,Jorge203,Chavarría957,NaN,...,-71.275475,545777.93,3282.24,1,1.0,1.0,22.0,0.0,0.0,0.0
2,30fb496e-b8d3-4d36-8e50-86acb3d1a223,1981-09-11,2020-07-02 23:24:55,999-89-5604,S99970660,X20079551X,Mr.,Octavio643,Schneider199,NaN,...,-72.784777,1039005.83,2796.44,1,1.0,1.0,38.0,0.0,0.0,0.0


In [70]:
hospitalized_ids.Id

0        26e5d262-6cdc-4274-a5cd-d7fd439e35f9
1        90031c21-e8a4-416e-b60b-23e871ee62dc
2        c58ae02e-235e-4db2-a9c3-6926e9c0c5fa
3        0e07d4b4-8c0a-4cd0-a439-8f3b96ab884d
4        ef272f3c-6cbd-401c-97b5-db5de666b6e5
                         ...                 
20397    184d0311-e033-4c97-8d66-3809ca91be94
20398    acff780f-ccd1-4cef-ba8f-40eb70972098
20399    65d3785a-6150-4b86-8842-c40ca654e3c4
20400    497bf8a5-2c19-4149-9a2e-f256c8165a7a
20401    8722685f-ce37-4520-ab95-22b6929526c2
Name: Id, Length: 20402, dtype: object

In [69]:
## num of days in icu  
icu_id = encounters[(encounters.CODE == 305351004)
                   &(encounters.PATIENT.isin(pos_covid_id.index))].PATIENT
covid_icu_days = DataFrame(encounters[encounters.PATIENT.isin(icu_id)])
covid_icu_days = covid_icu_days[['PATIENT','START','STOP','DESCRIPTION','ENCOUNTERCLASS']]
covid_icu_days['START']= covid_icu_days['START'].astype('datetime64[ns]')
covid_icu_days['STOP']= covid_icu_days['STOP'].astype('datetime64[ns]')
covid_icu_days['Covid_NUM_DAYS_ICU'] = (covid_icu_days['STOP']-covid_icu_days['START']).astype('timedelta64[D]').astype(float)
covid_icu_days = covid_icu_days[covid_icu_days.START.dt.year == 2020]
#covid_icu_days = covid_icu_days[covid_icu_days.Covid_NUM_DAYS_ICU != 0.0]
covid_icu_days = covid_icu_days.groupby(['PATIENT']).sum()
covid_icu_days
patients = pd.merge(patients,
                   covid_icu_days,
                   how = 'left',
                   left_on = 'Id',
                   right_on = 'PATIENT')
patients['Covid_NUM_DAYS_ICU'] = patients['Covid_NUM_DAYS_ICU'].fillna(0.0)
patients.head(3)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,COVID_STAT,Target_AD,Decease_2020,AGE,NUM_DAYS_HOPS,Covid_NUM_DAYS_ICU
0,93fa6213-73a1-4cce-b408-9986a4145801,1989-09-05,2020-07-02 23:26:50,999-87-8276,NaN,NaN,NaN,Roland928,Wolff180,NaN,...,42.398277,-71.289383,6520.55,0.00,1,1.0,0.0,30.0,0.0,0.0
1,53bbfa41-5cab-422e-a8a7-8fa3ae55ed71,1997-09-25,2020-07-02 23:24:55,999-96-1808,S99962046,X54333499X,Mr.,Jorge203,Chavarría957,NaN,...,42.691016,-71.275475,545777.93,3282.24,1,1.0,1.0,22.0,0.0,0.0
2,30fb496e-b8d3-4d36-8e50-86acb3d1a223,1981-09-11,2020-07-02 23:24:55,999-89-5604,S99970660,X20079551X,Mr.,Octavio643,Schneider199,NaN,...,42.174484,-72.784777,1039005.83,2796.44,1,1.0,1.0,38.0,0.0,0.0


In [ ]:
# Check:  icu patients is a subset of hospitalized patients
covid_hospitalized_days = covid_hospitalized_days.groupby(['PATIENT']).sum()
print(covid_hospitalized_days.shape)
a = pd.merge(covid_icu_days,
                   covid_hospitalized_days,
                   how = 'outer',
                   on = 'PATIENT')
print(a.shape)  # icu patients is a subset of hospitalized patients

In [75]:
# Participants are asked to submit a normalized confidence score, between 0 and 1, for each synthetic Veteran’s controlled ventilation status. 
# A value of 1 indicates confidence that the synthetic Veteran was ventilated as a result of COVID-19 illness, 
#while a value of 0 indicates confidence that the synthetic Veteran was not ventilated as a result of COVID-19 illness. 
#Veterans without COVID-19 should be given a value of 0.
vent_ids = procedures[(procedures.CODE == 26763009) 
                            & (procedures.PATIENT.isin(pos_covid_id.index))].PATIENT
patients['VENT'] = 0
patients.loc[patients.Id.isin(vent_ids),'VENT'] = 1

vent_procedures = procedures[procedures.PATIENT.isin(vent_ids)]  # find important features from here 

In [79]:
#Marital Status. M is married, S is single. Currently no support for divorce (D) or widowing (W)
patients['Marital_STAT'] = 0
patients.loc[patients.MARITAL == 'M','Marital_STAT'] = 1
patients.loc[(patients.MARITAL == 'S') 
             | (patients.MARITAL == 'D')
             | (patients.MARITAL == 'W'),'Marital_STAT'] = -1 

In [80]:
patients = patients.drop(['BIRTHDATE','DEATHDATE','SSN','FIRST','LAST','MAIDEN','PREFIX'],axis = 1)
patients = patients.drop(['ADDRESS'],axis = 1)
#importantfeatures = ['Id', , 'SUFFIX', 'MARITAL', 'RACE', 'ETHNICITY','GENDER', 'ZIP', 'LAT', 'LON','HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', ,'Marital_STAT'
#       'COVID_STAT','Decease_2020', 'Target_AD', 'AGE', 'NUM_DAYS_HOPS','Covid_NUM_DAYS_HOPS', 'Covid_NUM_DAYS_ICU', 'VENT'],
patients = patients.drop(['BIRTHPLACE', 'CITY', 'STATE', 'COUNTY', 'ZIP'],axis = 1)
patients.head(3)

,Id,DRIVERS,PASSPORT,SUFFIX,MARITAL,RACE,ETHNICITY,GENDER,LAT,LON,...,HEALTHCARE_COVERAGE,COVID_STAT,Target_AD,Decease_2020,AGE,NUM_DAYS_HOPS,Covid_NUM_DAYS_ICU,Covid_NUM_DAYS_HOPS,VENT,Marital_STAT
0,93fa6213-73a1-4cce-b408-9986a4145801,NaN,NaN,NaN,NaN,white,nonhispanic,M,42.398277,-71.289383,...,0.00,1,1.0,0.0,30.0,0.0,0.0,0.0,0,0
1,53bbfa41-5cab-422e-a8a7-8fa3ae55ed71,S99962046,X54333499X,NaN,NaN,asian,hispanic,M,42.691016,-71.275475,...,3282.24,1,1.0,1.0,22.0,0.0,0.0,0.0,0,0
2,30fb496e-b8d3-4d36-8e50-86acb3d1a223,S99970660,X20079551X,NaN,M,white,nonhispanic,M,42.174484,-72.784777,...,2796.44,1,1.0,1.0,38.0,0.0,0.0,0.0,0,1


In [127]:
patients.Target_AD.value_counts()

1.0    117959
Name: Target_AD, dtype: int64

--------------------

In [137]:
pos_covid_id.index

Index(['afa2680f-7f73-46d9-b0cd-2cf3db49724b',
       '26e5d262-6cdc-4274-a5cd-d7fd439e35f9',
       '90031c21-e8a4-416e-b60b-23e871ee62dc',
       '0ac2e031-ab88-41f9-ac5d-d7d588952222',
       '45138fb4-dc79-4aec-ba78-7a012d3edca7',
       'fba0823a-d6fc-402f-a6c3-9394f54b1870',
       '93ef4d47-2a1e-4af1-b792-96a6077b75e0',
       'ff524e84-ef48-4c59-8e16-ff28514271a1',
       '5c6dd9dd-effb-4ba6-a059-a60579b86ee1',
       'c58ae02e-235e-4db2-a9c3-6926e9c0c5fa',
       ...
       '3331c4dd-9378-499b-86e4-4d75d649b03a',
       'f1286319-f71e-4949-903d-f93aa6cd1be6',
       '9f430ccd-8c0d-4616-bbe4-78e220094c04',
       'c7ff291a-5968-452e-b79d-914a417a29d3',
       '9eab9057-9be3-4613-9799-166780bf7ab1',
       '43d96255-e3ab-43af-97e6-494681105115',
       '12f7b700-0f7a-4e09-8adf-7d7f9c0c2fb0',
       '8722685f-ce37-4520-ab95-22b6929526c2',
       '62e3e406-8375-4cf5-aae7-0d489e40e73c',
       '18b4b304-dddd-41e8-a13d-08a3285552be'],
      dtype='object', name='Id', length=73697)

In [ ]:
# consider about died patients after 2020 and pos_covid_ids
target_patients = DataFrame(patients.loc[patients.Decease_2020 == 0.0]
                              & patients.Id.isin(pos_covid_id))
target_patients.shape 

In [88]:
died_2020_ids = died_2020_patients['Id']
died_2020_ids

0         93fa6213-73a1-4cce-b408-9986a4145801
12        c58ae02e-235e-4db2-a9c3-6926e9c0c5fa
14        57021f90-0e28-49d0-bd18-5dd2b53519e1
15        c43badbe-12ae-4cb7-a23f-403abae9ee10
17        b19b626b-7e2a-4adc-98a8-dd17676b7d83
                          ...                 
117938    0e5f17d0-a351-4acb-ba28-94d6553857e0
117941    65d3785a-6150-4b86-8842-c40ca654e3c4
117954    8722685f-ce37-4520-ab95-22b6929526c2
117957    7b8af071-d744-4732-be07-4eb8932a7fe1
117958    89e8d8a3-f80d-4f4e-8458-94e5bc32a802
Name: Id, Length: 37960, dtype: object

## adding more features from other datasets

In [99]:
observations['DATE']= observations['DATE'].astype('datetime64[ns]')
covid_observations = DataFrame(observations.loc[observations.PATIENT.isin(deceased_covid19_ids.Id)])

covid_observation_his = DataFrame(covid_observations.loc[covid_observations.DATE.dt.year < 2020])
covid_observation_curr = DataFrame(covid_observations.loc[covid_observations.DATE.dt.year == 2020])
covid_observations_before_d = DataFrame(covid_observations[~(covid_observations.DESCRIPTION.str.contains('\[US Standard Certificate of Death]'))] )
covid_observations_d = DataFrame(covid_observations[covid_observations.DESCRIPTION.str.contains('\[US Standard Certificate of Death]')])

confirmed_date = DataFrame(covid_observations[covid_observations.VALUE == 'Detected (qualifier value)'])
confirmed_date = confirmed_date.groupby('PATIENT').first()
confirmed_date.rename(columns={'DATE': 'CONFIRMED_DATE'}, inplace=True) 
confirmed_date.drop(['ENCOUNTER','CODE','DESCRIPTION','VALUE','UNITS','TYPE'], axis = 1, inplace=True)

covid_observations_d.rename(columns={'DATE': 'DEATH_DATE'}, inplace=True)
covid_observations_d.rename(columns={'VALUE': 'DEATH_REASON'}, inplace=True)   # observation_DESCRIPTION: \[US Standard Certificate of Death]'_VALUE
covid_observations_d.drop(['ENCOUNTER','CODE','DESCRIPTION','UNITS','TYPE'], axis = 1, inplace=True)
CONFIR_TO_DEATH = pd.merge(confirmed_date,
                          covid_observations_d,
                          on = 'PATIENT',
                          how = 'left')

# find reason of death, except covid-19
CONFIR_TO_DEATH.DEATH_REASON.value_counts() # the values in test_observation set
CONFIR_TO_DEATH['Non-small_cell_lung_cancer'] = 0
CONFIR_TO_DEATH.loc[CONFIR_TO_DEATH.DEATH_REASON == 'Non-small cell lung cancer (disorder)','Non-small_cell_lung_cancer'] = 1
CONFIR_TO_DEATH['Neoplasm_of_prostate'] = 0
CONFIR_TO_DEATH.loc[CONFIR_TO_DEATH.DEATH_REASON == 'Neoplasm of prostate','Neoplasm_of_prostate'] = 1
CONFIR_TO_DEATH['Pneumonia'] = 0
CONFIR_TO_DEATH.loc[CONFIR_TO_DEATH.DEATH_REASON == 'Pneumonia','Pneumonia'] = 1
CONFIR_TO_DEATH['Myocardial_Infarction'] = 0
CONFIR_TO_DEATH.loc[CONFIR_TO_DEATH.DEATH_REASON == 'Myocardial Infarction','Myocardial_Infarction'] = 1
CONFIR_TO_DEATH['Small_cell_carcinoma_of_lung'] = 0
CONFIR_TO_DEATH.loc[CONFIR_TO_DEATH.DEATH_REASON == 'Small cell carcinoma of lung (disorder)','Small_cell_carcinoma_of_lung'] = 1
died_2020_patients = pd.merge(died_2020_patients,
                              CONFIR_TO_DEATH,
                              left_on = 'Id',
                              right_on = 'PATIENT',
                            how = 'left')
CONFIR_TO_DEATH 

,PATIENT,CONFIRMED_DATE,DEATH_DATE,DEATH_REASON,Non-small_cell_lung_cancer,Neoplasm_of_prostate,Pneumonia,Myocardial_Infarction,Small_cell_carcinoma_of_lung
0,00121fa6-3f13-48aa-885b-9d861e7794fd,2020-03-08,2020-03-30,COVID-19,0,0,0,0,0
1,001b86d2-9666-4f98-8f7d-56fbe819a318,2020-03-09,2020-03-26,COVID-19,0,0,0,0,0
2,00258336-19e2-473f-93f7-a65e7f7fa10d,2020-02-15,2020-03-09,COVID-19,0,0,0,0,0
3,002600d8-9bb1-4272-819a-2784d832e713,2020-02-16,2020-03-08,COVID-19,0,0,0,0,0
4,00445077-7f52-4ae2-9d37-2bf431a6f968,2020-03-05,2020-03-25,COVID-19,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
5563,ffb60a43-71f3-4a86-aaf9-6b45f6c77cdb,2020-03-12,2020-04-01,COVID-19,0,0,0,0,0
5564,ffce06c3-d95e-467a-bd43-b73525ee2074,2020-03-03,2020-03-27,COVID-19,0,0,0,0,0
5565,ffcfe390-a4a5-4cdd-bfd8-608e4848cc6a,2020-03-12,2020-03-29,COVID-19,0,0,0,0,0
5566,ffd133b0-2668-4e97-a17a-ae7bf68242f7,2020-03-14,2020-04-01,COVID-19,0,0,0,0,0


In [125]:
died_2020_patients.Non_small_cell_lung_cancer.value_counts()

0    37791
1      169
Name: Non_small_cell_lung_cancer, dtype: int64

In [124]:
# ? if num of 1 is so small change observations_d to observations. 
observations_d = DataFrame(observations[observations.DESCRIPTION.str.contains('\[US Standard Certificate of Death]')] )

observations_d['DATE'] = observations_d['DATE'].astype('datetime64[ns]')
observations_d = observations_d.loc[observations_d.DATE.dt.year == 2020]
#test_observations_d = DataFrame(test_observation[test_observation.DESCRIPTION.str.contains('\[US Standard Certificate of Death]')])
observations_d1 = DataFrame(observations_d[observations_d.VALUE =='Non-small cell lung cancer (disorder)'])
observations_d2 = DataFrame(observations_d[observations_d.VALUE =='Neoplasm of prostate'])
observations_d3 = DataFrame(observations_d[observations_d.VALUE =='Pneumonia'])
observations_d4 = DataFrame(observations_d[observations_d.VALUE =='Myocardial Infarction'])
observations_d5 = DataFrame(observations_d[observations_d.VALUE =='Small cell carcinoma of lung (disorder)'])
died_2020_patients['Non_small_cell_lung_cancer'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(observations_d1.PATIENT),'Non_small_cell_lung_cancer'] = 1
died_2020_patients['Neoplasm_of_prostate'] = 0
died_2020_patients['Neoplasm_of_prostate'] = np.where(died_2020_patients.Id.isin(observations_d2.PATIENT), 1, 0)
died_2020_patients.loc[died_2020_patients.Id.isin(observations_d2.PATIENT),'Neoplasm_of_prostate' ] = 1
died_2020_patients[died_2020_patients.Id.isin(observations_d2.PATIENT)]['Neoplasm_of_prostate'] = 1
died_2020_patients['Pneumonia'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(observations_d3.PATIENT),'Pneumonia'] = 1                                          #!! 
died_2020_patients['Myocardial_Infarction'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(observations_d4.PATIENT),'Myocardial_Infarction'] = 1
died_2020_patients['Small_cell_carcinoma_of_lung' ] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(observations_d5.PATIENT),'Small_cell_carcinoma_of_lung'] = 1
#died_2020_patients.drop(['PATIENT','CONFIRMED_DATE','DEATH_DATE','DEATH_REASON'],axis = 1, inplace = True)
#df['covid_status'] = np.where(df.PATIENT.isin(all_patient.tolist(), 1, 0)
#df.isin({'num_wings': [0, 3]})
#.value_counts().reset_index(drop=False).drop(
#    ['DESCRIPTION'], axis=1).rename(columns={'index': 'DESCRIPTION'}).reset_index().rename(columns={'index': 'label_description'})
#rpt[rpt['STK_ID'].isin(stk_list)] 

In [ ]:
observations_d1

In [ ]:
died_2020_patients[died_2020_patients.Id.isin(observations_d2.VALUE)]['Neoplasm_of_prostate'] = 1

In [ ]:
died_2020_patients[died_2020_patients['Neoplasm_of_prostate']==1]

In [ ]:
died_2020_patients[died_2020_patients.Id.isin(observations_d2.PATIENT)]

In [ ]:
observations_d2.PATIENT

In [ ]:
died_2020_patients.Neoplasm_of_prostate.value_counts()

----

In [ ]:
deathencounters = DataFrame(encounters.loc[encounters.PATIENT.isin(died_2020_ids)])
covid_death_encounters = DataFrame(encounters.loc[encounters.PATIENT.isin(deceased_covid19_ids.Id)])

encounters = encounters[encounters.ENCOUNTERCLASS != 'wellness']
deathencounters = deathencounters[deathencounters.ENCOUNTERCLASS != 'wellness']
covid_death_encounters = covid_death_encounters[covid_death_encounters.ENCOUNTERCLASS != 'wellness']

encounters = encounters[encounters.ENCOUNTERCLASS != 'outpatient']
deathencounters = deathencounters[deathencounters.ENCOUNTERCLASS != 'outpatient']
covid_death_encounters = covid_death_encounters[covid_death_encounters.ENCOUNTERCLASS != 'outpatient']

encounters['START'] = encounters['START'].astype('datetime64[ns]')
encounters['STOP'] = encounters['STOP'].astype('datetime64[ns]')
deathencounters['START'] = deathencounters['START'].astype('datetime64[ns]')
deathencounters['STOP'] = deathencounters['STOP'].astype('datetime64[ns]')
covid_death_encounters['START'] = covid_death_encounters['START'].astype('datetime64[ns]')
covid_death_encounters['STOP'] = covid_death_encounters['STOP'].astype('datetime64[ns]')

encounters_his = DataFrame(encounters.loc[encounters.STOP.dt.year < 2020])
encounters_curr = DataFrame(encounters.loc[encounters.START.dt.year == 2020])
deathencounters_his = DataFrame(deathencounters.loc[deathencounters.STOP.dt.year < 2020])
deathencounters_curr = DataFrame(deathencounters.loc[deathencounters.START.dt.year == 2020])
covid_death_encounters_his = DataFrame(covid_death_encounters.loc[covid_death_encounters.STOP.dt.year < 2020])
covid_death_encounters_curr = DataFrame(covid_death_encounters.loc[covid_death_encounters.START.dt.year == 2020])
# covid_death_encounters_curr[covid_death_encounters_curr.ENCOUNTERCLASS == 'emergency'].REASONDESCRIPTION.value_counts() == 'Chronic congestive heart failure (disorder)'
# covid_death_encounters_curr[covid_death_encounters_curr.ENCOUNTERCLASS == 'ambulatory'].REASONDESCRIPTION.value_counts() == Chronic congestive heart failure (disorder) 

encounters1 = DataFrame(encounters[encounters.ENCOUNTERCLASS == 'emergency'])
encounters2 = DataFrame(encounters[encounters.ENCOUNTERCLASS == 'ambulatory'])
encounters1 = encounters1[encounters1.REASONDESCRIPTION =='Chronic congestive heart failure (disorder)']
encounters2 = encounters2[encounters2.REASONDESCRIPTION =='Chronic congestive heart failure (disorder)']
encounters1.loc[encounters1.START.dt.year == 2020]
encounters2.loc[encounters2.START.dt.year == 2020]
died_2020_patients['Chronic_congestive_heart_failure'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(encounters1.PATIENT),'Chronic_congestive_heart_failure'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(encounters2.PATIENT),'Chronic_congestive_heart_failure'] = 1

In [ ]:
procedures['DATE'] = procedures['DATE'].astype('datetime64[ns]')
procedures_curr = DataFrame(procedures[procedures.DATE.dt.year == 2020])
died2020_procedures_curr = DataFrame(procedures_curr.loc[procedures_curr.PATIENT.isin(died_2020_ids)])
covid_died_procedures_curr = DataFrame(procedures_curr.loc[procedures_curr.PATIENT.isin(deceased_covid19_ids.Id)])
tprocedures['DATE'] = tprocedures['DATE'].astype('datetime64[ns]')
#tprocedures_curr = DataFrame(tprocedures[tprocedures.DATE.dt.year == 2020]) 
procedures1 = procedures[procedures.DESCRIPTION =='Oxygen Therapy']
procedures11 = procedures[procedures.DESCRIPTION =='Oxygen administration by mask (procedure)']
died_2020_patients['Oxygen_Therapy'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(procedures1.PATIENT),'Oxygen_Therapy'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(procedures11.PATIENT),'Oxygen_Therapy'] = 1

procedures2 = procedures[procedures.DESCRIPTION =='Plain chest X-ray (procedure)']
procedures22 = procedures[procedures.DESCRIPTION =='Plain chest X-ray']
died_2020_patients['Plain_chest_X_ray'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(procedures2.PATIENT),'Plain_chest_X_ray'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(procedures22.PATIENT),'Plain_chest_X_ray'] = 1

procedures3 = procedures[procedures.DESCRIPTION =='Mental health screening (procedure)']
died_2020_patients['Mental_health_screening'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(procedures3.PATIENT),'Mental_health_screening'] = 1


#tprocedures.DESCRIPTION.unique() !!!'Oxygen Therapy'  
#Oxygen administration by mask (procedure)     'Oxygen Therapy'     'Mental health screening (procedure)'                                    39709
#Placing subject in prone position (procedure) 
#Controlled ventilation procedure and therapy  initiation and management (procedure)    26761
#Hemodialysis (procedure)                                                               13995
#Plain chest X-ray (procedure)   

procedures4 = procedures[procedures.REASONDESCRIPTION =='Non-small cell carcinoma of lung  TNM stage 1 (disorder)']
procedures44 = procedures[procedures.REASONDESCRIPTION =='Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)']
procedures444 = procedures[procedures.REASONDESCRIPTION =='Suspected lung cancer (situation)']
procedures4444 = procedures[procedures.REASONDESCRIPTION =='Pulmonary emphysema (disorder)']
died_2020_patients['Pulmonary_diseases_cancer'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(procedures4.PATIENT),'Pulmonary_diseases_cancer'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(procedures44.PATIENT),'Pulmonary_diseases_cancer'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(procedures444.PATIENT),'Pulmonary_diseases_cancer'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(procedures4444.PATIENT),'Pulmonary_diseases_cancer'] = 1

procedures5 = procedures[procedures.REASONDESCRIPTION =='Chronic obstructive bronchitis (disorder)']
died_2020_patients['Chronic_bronchitis'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(procedures5.PATIENT),'Chronic_bronchitis'] = 1

procedures6 = procedures[procedures.REASONDESCRIPTION =='Myocardial Infarction']
procedures66 = procedures[procedures.REASONDESCRIPTION =='Atrial Fibrillation']
died_2020_patients['cardial_Atrial_diseases'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(procedures6.PATIENT),'cardial_Atrial_diseases'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(procedures66.PATIENT),'cardial_Atrial_diseases'] = 1

 
procedures7 = procedures[procedures.REASONDESCRIPTION =='Streptococcal sore throat (disorder)']
procedures77 = procedures[procedures.REASONDESCRIPTION =='Acute viral pharyngitis (disorder)']
died_2020_patients['throat_pharyngitis'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(procedures7.PATIENT),'throat_pharyngitis'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(procedures77.PATIENT),'throat_pharyngitis'] = 1
 

In [ ]:
immunizations['DATE'] = immunizations['DATE'].astype('datetime64[ns]')
immunizations_curr = DataFrame(immunizations[immunizations.DATE.dt.year == 2020])
died2020_immunizations_curr = DataFrame(immunizations_curr.loc[immunizations_curr.PATIENT.isin(died_2020_ids)])
covid_died_immunizations_curr = DataFrame(immunizations_curr.loc[immunizations_curr.PATIENT.isin(deceased_covid19_ids.Id)])
timmunizations['DATE'] = timmunizations['DATE'].astype('datetime64[ns]')

immunizations1 = immunizations[immunizations.DESCRIPTION =='pneumococcal polysaccharide vaccine  23 valent']
immunizations2 = immunizations[immunizations.DESCRIPTION =='Pneumococcal conjugate PCV 13']
died_2020_patients['immu_PCV'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(immunizations1.PATIENT),'immu_PCV'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(immunizations2.PATIENT),'immu_PCV'] = 1

In [ ]:
conditions['START'] = conditions['START'].astype('datetime64[ns]')
conditions['STOP'] = conditions['STOP'].astype('datetime64[ns]')
conditions_curr = DataFrame(conditions[conditions.STOP.dt.year == 2020])
died2020_conditions_curr = DataFrame(conditions.loc[conditions.PATIENT.isin(died_2020_ids)])
covid_died_conditions_curr = DataFrame(conditions.loc[conditions.PATIENT.isin(deceased_covid19_ids.Id)])
#tconditions['DATE'] = tconditions['DATE'].astype('datetime64[ns]')

conditions1 = conditions[conditions.DESCRIPTION =='Respiratory distress (finding)']
conditions2 = conditions[conditions.DESCRIPTION =='Pneumonia (disorder)']
conditions3 = conditions[conditions.DESCRIPTION =='Hypoxemia (disorder)']                            
died_2020_patients['Respiratory_distress'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(conditions1.PATIENT),'Respiratory_distress'] = 1
died_2020_patients['Pneumonia_'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(conditions2.PATIENT),'Pneumonia_'] = 1
died_2020_patients['Hypoxemia'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(conditions3.PATIENT),'Hypoxemia'] = 1
  

In [ ]:
medications['START'] = medications['START'].astype('datetime64[ns]')
medications['STOP'] = medications['STOP'].astype('datetime64[ns]')
medications_curr = DataFrame(medications[medications.STOP.dt.year == 2020])
died2020_medications_curr = DataFrame(medications_curr.loc[medications_curr.PATIENT.isin(died_2020_ids)])
covid_died_medications_curr = DataFrame(medications_curr.loc[medications_curr.PATIENT.isin(deceased_covid19_ids.Id)])

medications1 = medications[medications.DESCRIPTION =='Hypertension']
died_2020_patients['Hypertension'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(medications1.PATIENT),'Hypertension'] = 1

medications2 = medications[medications.DESCRIPTION =='Non-small cell carcinoma of lung  TNM stage 1 (disorder)']
medications22 = medications[medications.DESCRIPTION =='Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)']  
died_2020_patients['carcinoma_of_lung'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(medications2.PATIENT),'carcinoma_of_lung'] = 1
died_2020_patients.loc[died_2020_patients.Id.isin(medications22.PATIENT),'carcinoma_of_lung'] = 1

In [ ]:
careplans['START'] = careplans['START'].astype('datetime64[ns]')
careplans['STOP'] = careplans['STOP'].astype('datetime64[ns]')
careplans_curr = DataFrame(careplans[careplans.STOP.dt.year == 2020])
died2020_careplans_curr = DataFrame(careplans_curr.loc[careplans_curr.PATIENT.isin(died_2020_ids)])
covid_died_careplans_curr = DataFrame(careplans_curr.loc[careplans_curr.PATIENT.isin(deceased_covid19_ids.Id)])

careplans1 = careplans[careplans.DESCRIPTION =='Respiratory therapy'] 
died_2020_patients['Respiratory_therapy'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(careplans1.PATIENT),'Respiratory_therapy'] = 1


careplans2 = careplans[careplans.DESCRIPTION =='Chronic obstructive pulmonary disease clinical management plan'] 
died_2020_patients['Chronic_pulmonary'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(careplans2.PATIENT),'Chronic_pulmonary'] = 1

careplans3 = careplans[careplans.DESCRIPTION =='Asthma self management'] 
died_2020_patients['Asthma'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(careplans3.PATIENT),'Asthma'] = 1
 
careplans4 = careplans[careplans.DESCRIPTION =='Cancer care plan'] 
died_2020_patients['Cancer_care_plan'] = 0
died_2020_patients.loc[died_2020_patients.Id.isin(careplans4.PATIENT),'Cancer_care_plan'] = 1
#covidonly_died_careplans_curr = DataFrame(covid_died_careplans_curr.loc[(covid_died_careplans_curr.REASONDESCRIPTION == 'Suspected COVID-19')
#                                                                       |(covid_died_careplans_curr.REASONDESCRIPTION == 'COVID-19')])

In [ ]:
died_2020_patients.loc[died_2020_patients.Id.isin(careplans3.PATIENT)] 

-----------

# modelling

In [ ]:
import matplotlib.pyplot as plt # drawing graphs
from sklearn.tree import DecisionTreeClassifier # a classification tree
from sklearn.tree import plot_tree # draw a classification tree
from sklearn.model_selection import train_test_split # split  data into training and testing sets
from sklearn.model_selection import cross_val_score # cross validation
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix # creates a confusion matrix
from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix
X = patients.drop('target1',axis = 1).copy()
y = patients['target1'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)
clf_dt = DecisionTreeClassifier(random_state=42)
clf_dt = clf_dt.fit(X_train, y_train)
plot_confusion_matrix(clf_dt, X_test, y_test, display_labels=["doesn't death", "Has death"])
path = clf_dt.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
ccp_alphas = ccp_alphas[:-1]

clf_dts = []
for ccp_alpha in ccp_alphas:
    clf_dt = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf_dt.fit(X_train, y_train)
    clf_dts.append(clf_dt)
    
train_scores = [clf_dt.score(X_train, y_train) for clf_dt in clf_dts] # the accuracy of the classifications 
test_scores = [clf_dt.score(X_test, y_test) for clf_dt in clf_dts]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test", drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
s = np.array(conditions.DESCRIPTION.str.contains('COVID-19', regex=1))
conditions['symptomCovid_19'] = s
Covid_19symptom=dict({False:0,True:1 })
conditions['symptomCovid-19'] = conditions.symptomCovid_19.replace(Covid_19symptom)

covid_patient_ids = conditions[conditions.CODE == 840539006].PATIENT.unique()
allpatients=conditions['PATIENT'].unique()
covid_patientsdict={}
for index, i in enumerate(allpatients): 
    if i in covid_patient_ids:
        covid_patientsdict[i]=1
    else:
        covid_patientsdict[i]=0
        
allpatients=conditions['PATIENT'].unique()
conditions['covid_patient_ids'] = np.where(conditions.PATIENT.isin(allpatients.tolist()))